<a href="https://colab.research.google.com/github/nagabathula/C255-DataMIning/blob/main/Assignment5_ANN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ANN Algothm
https://www.kaggle.com/datasets/zygmunt/goodbooks-10k?datasetId=1938

In [ ]:
!pip install faiss-gpu
!pip install annoy
!pip install nmslib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Imports
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
import faiss
from google.colab import drive
import annoy
import nmslib

pd.set_option('display.max_columns', None)

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
RANDOM_STATE = 10
N_FACTOR = 32
USER_ID = 75

In [ ]:
# Read datasets
books = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/books.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv')

In [ ]:
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [ ]:
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [ ]:
df = pd.merge(ratings, books, how='left', on='book_id')

In [ ]:
# Filter for books only in english
english = ['eng', 'en-US', 'en-GB',]
df = df[df.language_code.isin(english)]

df = df[['book_id', 'user_id', 'original_title', 'rating']]

In [ ]:
# Check for nulls
df.isna().sum()

book_id              0
user_id              0
original_title    1500
rating               0
dtype: int64

In [ ]:
# Drop rows where book title is null
df.dropna(inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

,book_id,user_id,original_title,rating
0,1,314,Harry Potter and the Half-Blood Prince,5
1,1,439,Harry Potter and the Half-Blood Prince,3
2,1,588,Harry Potter and the Half-Blood Prince,5
3,1,1169,Harry Potter and the Half-Blood Prince,4
4,1,1185,Harry Potter and the Half-Blood Prince,4


In [ ]:
df.shape

(66306, 4)

In [ ]:
# Make coo sparse matrix 
rating_matrix = coo_matrix((df['rating'], (df['user_id'], df['book_id'])))

# Decompose sparse matrix using NMF where every vector is of length N_FACTOR
model = NMF(n_components=N_FACTOR, init='random', random_state=RANDOM_STATE)
user_matrix = model.fit_transform(rating_matrix)
book_matrix = model.components_.T

In [ ]:
# Search for topk books near the book read by user USER_ID
def searchNearestNeighbours(index, topk):
  user_vec = user_matrix[USER_ID].astype('float32')
  distance, indices = index.search(np.array([user_vec]), topk)
  return indices

In [ ]:
def printNearestNeighbours(indices):
  print(f'The most similar books to "{df.original_title.iloc[USER_ID]}" are:\n')
  for i in indices[0]:
    print(df.original_title.iloc[i])

### Exhaustive search

In [ ]:
# Create index using IndexFlatIP which uses cosine distance
book_index = faiss.IndexFlatIP(N_FACTOR)
book_index.add(book_matrix.astype('float32'))

In [ ]:
indices = searchNearestNeighbours(book_index, 10)
printNearestNeighbours(indices)

The most similar books to "Harry Potter and the Half-Blood Prince" are:

Harry Potter and the Half-Blood Prince
The Hitchhiker's Guide to the Galaxy
Harry Potter and the Goblet of Fire
Harry Potter and the Prisoner of Azkaban
Down Under
Harry Potter and the Goblet of Fire
Harry Potter and the Half-Blood Prince
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Prisoner of Azkaban
What to Expect the First Year


### Product Quantization

In [ ]:
# Create index using IndexIVFPQ
quantizer = faiss.IndexFlatL2(N_FACTOR)
index = faiss.IndexIVFPQ(quantizer, N_FACTOR, 50, 8, 8)
index.train(book_matrix.astype('float32'))
index.add(book_matrix.astype('float32'))

In [ ]:
indices = searchNearestNeighbours(index, 10)
printNearestNeighbours(indices)

The most similar books to "Harry Potter and the Half-Blood Prince" are:

Deception Point
Digging to America
Zodiac
Liar's Poker: Rising Through the Wreckage on Wall Street
1776
Moon Palace
Freakonomics: A Rogue Economist Explores the Hidden Side of Everything
The Psychology of Everyday Things
The Good Earth
Play It as It Lays


### Trees

In [ ]:
# Create index using AnnoyIndex
index = annoy.AnnoyIndex(N_FACTOR)
for i, vec in enumerate(book_matrix):
    index.add_item(i, vec.tolist())
index.build(n_trees = 20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


True

In [ ]:
user_vec = user_matrix[USER_ID].astype('float32')
indices = index.get_nns_by_vector(user_vec.tolist(), 10)    

In [ ]:
printNearestNeighbours([indices])

The most similar books to "Harry Potter and the Half-Blood Prince" are:

Harry Potter and the Prisoner of Azkaban
The Hitchhiker's Guide to the Galaxy
Sailing Alone Around the Room: New and Selected Poems
Deception Point
Moon Palace
The Known World
Das Parfum. Die Geschichte eines Mörders
Digging to America
Dirk Gently's Holistic Detective Agency
Heretics of Dune


### Hierarchical Navigable Small World Graphs

In [ ]:
# Create index for hnsw
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(book_matrix.astype('float32'))
index.createIndex()

In [ ]:
user_vec = user_matrix[USER_ID].astype('float32')
indices = index.knnQuery(user_vec, k=10)

In [ ]:
printNearestNeighbours(indices)

The most similar books to "Harry Potter and the Half-Blood Prince" are:

Harry Potter and the Half-Blood Prince
Down Under
What to Expect the First Year
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Goblet of Fire
Harry Potter and the Half-Blood Prince
Harry Potter and the Goblet of Fire
The Hitchhiker's Guide to the Galaxy
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Order of the Phoenix
